# What's new in scipp

This page highlights feature additions and discusses major changes from recent releases.
For a full list of changes see the [Release Notes](https://scipp.github.io/about/release-notes.html).

In [ ]:
import numpy as np
import scipp as sc

## General

### Unique dimensions and slicing of 1-D objects

<div class="alert alert-info">

**New in 0.9**

The new `dim` property checks whether an object is 1-D, and returns the only dimension label.
An exception is raised if the object is not 1-D.
</div>

Example:

In [ ]:
x = sc.linspace(dim='x', start=0, stop=1, num=4)
x.dim

<div class="alert alert-info">

**New in 0.11**

1-D objects can now be sliced without specifying a dimension.
</div>

Example:

In [ ]:
x[-1]

If an object is not 1-D then `DimensionError` is raised:

In [ ]:
var2d = sc.concat([x,x], 'y')
var2d[0]

### Logging support

<div class="alert alert-info">

**New in 0.9**

Scipp now provides a logger, and a pre-configured logging widget for Jupyter notebooks.
See [Logging](../reference/logging.ipynb).
    
</div>

### Bound method equivalents to many free functions

<div class="alert alert-info">

**New in 0.8**

Many functions that have been available as free functions can now be used also as methods of variables and data arrays.
See the [documentation for individual classes](../reference/classes.rst#classes) for a full list.

</div>

Example:

In [ ]:
var = sc.arange(dim="x", unit="m", start=0, stop=12)
var.sum()  # Previously sc.sum(var)

Note that `sc.sum(var)` will continue to be supported as well.

### Unified conversion of unit and dtype

<div class="alert alert-info">

**New in 0.11**

Variables and data arrays have a new method, `to`, for conversion of dtype, unit, or both.
This can be used to replace uses of `to_unit` and `astype`.

</div>

Example:

In [ ]:
var = sc.arange(dim='x', start=0, stop=4, unit='m')
var

Use the `unit` keyword argument to convert to a different unit:

In [ ]:
var.to(unit='mm')

Use the `dtype` keyword argument to convert to a different dtype:

In [ ]:
var.to(dtype='float64')

If both `unit` and `dtype` are provided, the implementation attempts to apply the two conversions in optimal order to reduce or avoid the effect of rounding/truncation errors:

In [ ]:
var.to(dtype='float64', unit='km')

### Operations

#### Creation functions

<div class="alert alert-info">

**New in 0.11**
    
Creation functions for datetimes where added:

- Added `epoch`, `datetime` and `datetimes`.

</div>

In [ ]:
sc.datetime('now', unit='ms')

In [ ]:
times = sc.datetimes(dims=['time'], values=['2022-01-11T10:24:03', '2022-01-11T10:24:03'])
times

The new `epoch` function is useful for obtaining the time since epoch, i.e., a time difference (`dtype='int64'`) instead of a time point (`dtype='datetime64'`):

In [ ]:
times - sc.epoch(unit=times.unit)

#### `from_pandas` and `from_xarray`

<div class="alert alert-info">

**New in 0.8**

- `from_pandas` for converting `pandas.Dataframe` to `scipp.Dataset`.
- `from_xarray` for converting `xarray.DataArray` or `xarray.Dataset` to `scipp.DataAray` or `scipp.Dataset`, respectively.

Both functions are available in the `compat` submodule.

</div>

### Reduction operations

#### Internal precision in summation operations

<div class="alert alert-info">

**New in 0.9**

Reduction operations such as `sum` of single-precision (`float32`) data now use double-precision (`float64`) internally to reduce the effects of rounding errors.

</div>

#### Reductions over multiple inputs using `reduce`

<div class="alert alert-info">

**New in 0.9**

The new `reduce` function can be used for reduction operations that do not operate along a dimension of a scipp object but rather across a list or tuple of multiple scipp objects.
The mechanism is a 2-step approach, with a syntax similar to `groupby`:

</div>

In [ ]:
a = sc.linspace(dim="x", start=0.0, stop=1.0, num=4)
b = sc.linspace(dim="x", start=0.2, stop=0.8, num=4)
c = sc.linspace(dim="x", start=0.2, stop=1.2, num=4)
sc.reduce([a, b, c]).sum()

In [ ]:
reducer = sc.reduce([a, b, c])
reducer.min()

In [ ]:
reducer.max()

### Shape operations

#### `concat` replacing `concatenate`

<div class="alert alert-info">

**New in 0.9**

`concat` is replacing `concatenate` (which is deprecated now and will be removed in 0.10).
It supports a list of inputs rather than just 2 inputs.

</div>

In [ ]:
a = sc.scalar(1.2)
b = sc.scalar(2.3)
c = sc.scalar(3.4)
sc.concat([a, b, c], "x")

### Vectors and matrices

#### General

<div class="alert alert-info">

**New in 0.11**
    
`scipp.spatial` has been restructured and extended:

- New data types for spatial transforms were added:
  - `vector3` (renamed from `vector3_float64`)
  - `rotation3` (3-D rotation defined using quaternion coeffiecients)
  - `translation3` (translation in 3-D)
  - `linear_transform3` (previously `matrix_3_float64`, 3-D linear transform with, e.g., rotation and scaling)
  - `affine_transform3` (affine transform in 3-D, combination of a linear transform and a translation, defined using 4x4 matrix)
- The [scipp.spatial](https://scipp.github.io/generated/modules/scipp.spatial.html) submodule was extended with a number of new creation functions, in particular for the new dtypes.
- `matrix` and `matrices` for creating "matrices" have been deprecated. Use `scipp.spatial.linear_transform` and `scipp.spatial.linear_transforms` instead.

</div>

Note that the `scipp.spatial` subpackage must be imported explicitly:

In [ ]:
from scipp import spatial
linear = spatial.linear_transform(value=[[1,0,0],[0,2,0],[0,0,3]])
linear

In [ ]:
trans = spatial.translation(value=[1,2,3], unit='m')
trans

Multiplication can be used to combine the various transforms:

In [ ]:
linear * trans

Note that in the case of `affine_transform3` the unit refers to the translation part.
A unit for the linear part is currently not supported.

### Coordinate transformations

<div class="alert alert-info">

**New in 0.8**

The `transform_coords` function has been added (also available as method of data arrays and datasets).
It is a tool for transforming one or more input coordinates into one or more output coordinates. It automatically handles:

- Renaming of dimensions, if dimension-coordinates are transformed.
- Change of coordinates to attributes to avoid interference of coordinates consumed by the transformation in follow-up operations.
- Conversion of event-coordinates of binned data, if present.

See [Coordinate transformations](../user-guide/coordinate-transformations.ipynb) for a full description.

</div>

### Physical constants

<div class="alert alert-info">

**New in 0.8**
    
The `scipp.constants` (in the style of `scipy.constants`) submodule was added, providing physical constants from CODATA 2018.
For full details see the [module's documentation](../generated/modules/scipp.constants.rst).

</div>

Examples:

In [ ]:
from scipp.constants import hbar, m_e, physical_constants

In [ ]:
hbar

In [ ]:
m_e

In [ ]:
physical_constants("speed of light in vacuum")

In [ ]:
physical_constants("neutron mass", with_variance=True)

## SciPy compatibility layer

<div class="alert alert-info">

**New in 0.11**
    
A number of subpackages providing wrappers for a *subset* of functions from the corresponding packages in SciPy was added:
    
- [scipp.integrate](../generated/modules/scipp.integrate.rst) providing `simpson` and `trapezoid`.
- [scipp.interpolate](../generated/modules/scipp.interpolate.rst) providing `interp1d`.
- [scipp.optimize](../generated/modules/scipp.optimize.rst) providing `curve_fit`.
- [scipp.signal](../generated/modules/scipp.signal.rst) providing `butter` and `sosfiltfilt`.

</div>

Please refer to the function documentation for working examples.

## Performance

<div class="alert alert-info">

**New in 0.9**

- `sc.lookup(histogram, dim)[var]` is now faster if `histogram` is very long and is integer-valued.
  This is relevant in a number of event-filtering operations.

</div>